In [157]:
# Librairies basiques d'exploitation et visualization de données
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [158]:
import random # Pour pouvoir mélanger notre jeu de données

In [187]:
# Modules pour le pré-processing
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA as ACP
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler

# Module pour l'affichage
from sklearn import set_config
set_config(display="diagram")

# Librairie pour la lecture et enregistrement des modèles
import pickle

In [160]:
import warnings
warnings.filterwarnings("ignore")

In [161]:
# On crée un objet qui enlève les colonnes qui ne nous intéressent pas.
class DropUnwantedColumns(TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        columns_to_drop = [col for col in self.columns if col in X.columns]
        return X.drop(columns_to_drop, axis=1)

In [164]:
# On crée un objet qui joint de l'open data 
class OpenData_Departement(TransformerMixin):
    def __init__(self, path, var_join):
        if ".xlsx" in path:
            self.df_open_data = pd.read_excel(path)
        elif ".csv" in path: 
            self.df_open_data = pd.read_csv(path)
        self.var_join = var_join
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_merged = pd.merge(X, self.df_open_data, on=self.var_join, how="left")
        return X_merged
        

In [165]:
pd.set_option('display.max_columns', None)
# On importe le jeu de données de notre échantillon test.
sub  = pd.read_csv("submissions.csv")
sub.head()

,Date mutation,Nature mutation,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,ID
0,03/01/2022,Vente,13.0,NaN,RUE,2280,DE LA LIBERTE,1000.0,BOURG-EN-BRESSE,1,53,NaN,AM,102,NaN,7.0,"24,10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,24.0,1.0,NaN,NaN,NaN,1
1,03/01/2022,Vente,98.0,NaN,RTE,0055,DE LA DOMBES,1480.0,SAVIGNEUX,1,398,NaN,ZE,187,NaN,1.0,"123,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,140.0,3.0,NaN,NaN,NaN,2
2,06/01/2022,Vente,282.0,NaN,RTE,0130,DE POISATON,1560.0,MANTENAY-MONTLIN,1,230,NaN,ZM,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,108.0,5.0,S,NaN,649.0,3
3,05/01/2022,Vente,7.0,NaN,RUE,0031,DU CORNIER,1150.0,VAUX-EN-BUGEY,1,431,NaN,A,1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,85.0,4.0,S,NaN,310.0,4
4,06/01/2022,Vente,7.0,NaN,RUE,3125,DES PINS,1000.0,BOURG-EN-BRESSE,1,53,NaN,CS,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,99.0,5.0,S,NaN,765.0,5


In [370]:
# On vérifie son format.
sub.shape

(378041, 35)

In [172]:
# On encode la variable Nature culture où on assigne un booléen si la valeur est nulle.
sub['exterieur'] = np.where(sub['Nature culture'].isnull(), False, True)
sub.head()

,Date mutation,Nature mutation,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,ID,exterieur
0,03/01/2022,Vente,13.0,NaN,RUE,2280,DE LA LIBERTE,1000.0,BOURG-EN-BRESSE,1,53,NaN,AM,102,NaN,7.0,"24,10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,24.0,1.0,NaN,NaN,NaN,1,False
1,03/01/2022,Vente,98.0,NaN,RTE,0055,DE LA DOMBES,1480.0,SAVIGNEUX,1,398,NaN,ZE,187,NaN,1.0,"123,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,140.0,3.0,NaN,NaN,NaN,2,False
2,06/01/2022,Vente,282.0,NaN,RTE,0130,DE POISATON,1560.0,MANTENAY-MONTLIN,1,230,NaN,ZM,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,108.0,5.0,S,NaN,649.0,3,True
3,05/01/2022,Vente,7.0,NaN,RUE,0031,DU CORNIER,1150.0,VAUX-EN-BUGEY,1,431,NaN,A,1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,85.0,4.0,S,NaN,310.0,4,True
4,06/01/2022,Vente,7.0,NaN,RUE,3125,DES PINS,1000.0,BOURG-EN-BRESSE,1,53,NaN,CS,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,99.0,5.0,S,NaN,765.0,5,True


In [166]:
# On récupère la base de données des communes/département
departement = pd.read_csv("communes-departement-region.csv")
departement.head()

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes


In [167]:
# On ne garde que les variables qui nous serviront à cartographier nos données
departement  = departement.loc[:,["code_departement","nom_departement","nom_region"]]
# On enlève les duplicatas
departement = departement.drop_duplicates()
# On enlève les valeurs manquantes
departement = departement.dropna()
# On récupère donc un tableau avec juste les départements, régions et code département.
departement

,code_departement,nom_departement,nom_region
0,1,Ain,Auvergne-Rhône-Alpes
457,2,Aisne,Hauts-de-France
1287,3,Allier,Auvergne-Rhône-Alpes
1608,4,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur
1852,5,Hautes-Alpes,Provence-Alpes-Côte d'Azur
...,...,...,...
38725,971,Guadeloupe,Guadeloupe
38763,972,Martinique,Martinique
38801,973,Guyane,Guyane
38826,974,La Réunion,La Réunion


In [173]:
# On change les types de données des codes départements avant la jointure.
departement["code_departement"] = departement["code_departement"].astype(str)
sub["Code departement"] = sub["Code departement"].astype(str)

In [174]:
# On joint le jeu de données des départements avec notre échantillon test.
sub2 = pd.merge(departement, sub, left_on=["code_departement"], right_on=["Code departement"], how='inner')
sub2.head()

,code_departement,nom_departement,nom_region,Date mutation,Nature mutation,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,ID,exterieur
0,1,Ain,Auvergne-Rhône-Alpes,03/01/2022,Vente,13.0,NaN,RUE,2280,DE LA LIBERTE,1000.0,BOURG-EN-BRESSE,1,53,NaN,AM,102,NaN,7.0,"24,10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,24.0,1.0,NaN,NaN,NaN,1,False
1,1,Ain,Auvergne-Rhône-Alpes,03/01/2022,Vente,98.0,NaN,RTE,0055,DE LA DOMBES,1480.0,SAVIGNEUX,1,398,NaN,ZE,187,NaN,1.0,"123,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,140.0,3.0,NaN,NaN,NaN,2,False
2,1,Ain,Auvergne-Rhône-Alpes,06/01/2022,Vente,282.0,NaN,RTE,0130,DE POISATON,1560.0,MANTENAY-MONTLIN,1,230,NaN,ZM,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,108.0,5.0,S,NaN,649.0,3,True
3,1,Ain,Auvergne-Rhône-Alpes,05/01/2022,Vente,7.0,NaN,RUE,0031,DU CORNIER,1150.0,VAUX-EN-BUGEY,1,431,NaN,A,1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,85.0,4.0,S,NaN,310.0,4,True
4,1,Ain,Auvergne-Rhône-Alpes,06/01/2022,Vente,7.0,NaN,RUE,3125,DES PINS,1000.0,BOURG-EN-BRESSE,1,53,NaN,CS,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,99.0,5.0,S,NaN,765.0,5,True


In [175]:
# On vérifie que l'on a pas perdu de lignes.
sub2.shape

(378041, 38)

In [378]:
# On crée une pipeline de cleaning
cleaning = Pipeline(
    steps=[
        # On enlèves les variables qui ne nous intéressent pas
        ("droping", DropUnwantedColumns([
        'B/T/Q', '1er lot','Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot','Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       "Nature culture", "Nature culture speciale", "Prefixe de section", "Section", "Nature mutation", "Type de voie", "Voie", "No voie", "Code voie",
       "Code departement", "No plan", "No Volume", "Identifiant local", "ID", "Date mutation" ])),
       # On ajoute le pourcentage de population active par département
       ("pop_active", OpenData_Departement("pop_active.xlsx", "nom_departement")),
       # On ajoute le salaire moyen horaire par département
       ("salaire", OpenData_Departement("base-cc-bases-tous-salaries-2021.xlsx", "nom_departement")),
       # On ajoute le nombre d'écoles élémentaires par département
       ("ecole", OpenData_Departement("ecoles2.xlsx", "nom_departement")),
       # On ajoute le prix au m² (moyen, Q1, Q3)
       ("prix_m2", OpenData_Departement("m2.csv", "nom_departement")),
       # On ajoute le nombre de ventes
       ("nbre_ventes", OpenData_Departement("nbre_ventes.csv", "nom_departement")),
       # On enlève les dernières colonnes qui n'ont pas été utilisé pour l'apprentissage
       ("dropping2", DropUnwantedColumns(["code_departement_x", "year", "month", "code_departement_y", "Code postal",	"Commune",	"Code commune", "nom_departement", "nom_region"]))
         ]
)

cleaning

Pipeline(steps=[('droping',
                 <__main__.DropUnwantedColumns object at 0x0000027514C7B190>),
                ('pop_active',
                 <__main__.OpenData_Departement object at 0x0000027514C7B880>),
                ('salaire',
                 <__main__.OpenData_Departement object at 0x00000275034E4BE0>),
                ('ecole',
                 <__main__.OpenData_Departement object at 0x0000027506FF1640>),
                ('prix_m2',
                 <__main__.OpenData_Departement object at 0x0000027514BF5610>),
                ('nbre_ventes',
                 <__main__.OpenData_Departement object at 0x00000275149B9970>),
                ('dropping2',
                 <__main__.DropUnwantedColumns object at 0x000002748D0EC430>)])

In [379]:
# On crée un nouveau dataframe nettoyé.
sub3 = cleaning.fit_transform(sub2)

In [380]:
sub3.head()

,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain,exterieur,pop_active,salaire_moyen,nb_etab_elem,mean_prixm2,q1_prixm2,q3_prixm2,Total_Mutations
0,1,Appartement,24.0,1.0,NaN,False,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389.0
1,1,Appartement,140.0,3.0,NaN,False,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389.0
2,0,Maison,108.0,5.0,649.0,True,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389.0
3,0,Maison,85.0,4.0,310.0,True,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389.0
4,0,NaN,99.0,5.0,765.0,True,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389.0


In [182]:
# Pour toutes les colonnes quantitatives, on remplace les valeurs nullles par zéro.
for col in ['Nombre de lots', 'Surface reelle bati',
       'Nombre pieces principales', 'Surface terrain', 'pop_active', 'salaire_moyen', 'nb_etab_elem', 'mean_prixm2',
       'q1_prixm2', 'q3_prixm2', 'Total_Mutations']:
    sub3[col]= sub3[col].fillna(value=0)

In [155]:
# On importe notre arbre de décision utilisé pour classifier les types de locaux
with open('tree_classif_typelocal3.pkl', 'rb') as file:
    model = pickle.load(file)

In [183]:
# Pour chaque ligne du jeu de données
for index, row in sub3.iterrows():
    # Si la valeur de type local est nulle,
    if pd.isnull(row['Type local']):
        temp_row = row.fillna(0)  
        # On prédit le type de local
        predicted_value = model.predict([temp_row])[0]
        # Et on le positionne dans notre variable Type local
        sub3.at[index, 'Type local'] = predicted_value

In [184]:
sub3.head()

,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain,exterieur,pop_active,salaire_moyen,nb_etab_elem,mean_prixm2,q1_prixm2,q3_prixm2,Total_Mutations
0,1,Appartement,24.0,1.0,0.0,False,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389.0
1,1,Appartement,140.0,3.0,0.0,False,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389.0
2,0,Maison,108.0,5.0,649.0,True,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389.0
3,0,Maison,85.0,4.0,310.0,True,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389.0
4,0,Maison,99.0,5.0,765.0,True,320682.0,15.336249,405,2235.777587,1662.166667,2653.858156,12389.0


In [188]:
class Qual_Standardize(TransformerMixin):
    # On standardize les valeurs qualitatives en utilisant la racine carré de p_k.
    def __init__(self):
        self.p_k = None
    # Notre fit calcule la valeur p_k nécessaire à la transformation.
    def fit(self, X, y=None):
        qual_int = X.astype(int)
        # On calcule la valeur p_k comme la proportion de True dans la colonne
        self.p_k = np.sum(qual_int, axis=0) / qual_int.shape[0]
        return self
    def transform(self, X, y=None):
        qual_int = X.astype(int)    
        # On transforme chaque valeur du tableau disjonctif complet par la racine carré de p_k
        qual_trans = qual_int / (np.sqrt(self.p_k))

        return qual_trans

In [345]:
# On importe la pipeline de preprocessing et le modèle de régression utilisé pour prédire le prix des dépendances.
with open("dependance_preprocessing.pkl", 'rb') as file_prep:
            preprocessing_dep = pickle.load(file_prep) 
with open("dependance_model.pkl", 'rb') as file_reg:
            regression_dep = pickle.load(file_reg)
# On importe l'arbre de décision utilisé pour prédire le prix des locaux.
with open("local_model.pkl", 'rb') as file_reg:
            tree_local = pickle.load(file_reg)
# On importe la pipeline de preprocessing et le modèle de régression utilisé pour prédire le prix des maisons.
with open("maison_preprocessing.pkl", 'rb') as file_prep:
            preprocessing_maison = pickle.load(file_prep)
with open("maison_model.pkl", 'rb') as file_reg:
            regression_maison = pickle.load(file_reg)
# On importe la pipeline de preprocessing et le modèle de régression utilisé pour prédire le prix des appartements.
with open("appartement_preprocessing.pkl", 'rb') as file_prep:
            preprocessing_appartement = pickle.load(file_prep)
with open("appartement_model.pkl", 'rb') as file_reg:
            regression_appartement = pickle.load(file_reg)

In [312]:
# On assigne "Local" pour abbrévier.
sub3.loc[sub3["Type local"] == "Local industriel. commercial ou assimilé", "Type local"] = "Local"

In [285]:
# On s'assure qu'"extérieur" est bien considéré de type "booléen".
sub3["exterieur"] = sub3["exterieur"].astype(bool)

In [366]:
# On crée une liste vide
predictions = []


# Pour chaque ligne de notre jeu de données,
for index, row in sub3.iterrows():
    # On extrait le type de biens
    type_local = row.loc["Type local"]
    
    # Si c'est un local
    if type_local == "Local":
        # On enlève la variable "Type local"
        row.drop("Type local", axis=0, inplace=True)
        # On enlève la variable "Nombre de lots"
        row.drop("Nombre de lots", axis=0, inplace=True)        
        # On enlève la variable "Nombre pieces principales"
        row.drop("Nombre pieces principales", axis=0, inplace=True)
        # On calcule une estimation du prix du bien
        new_value = row["Surface reelle bati"] * row["q1_prixm2"]
        # On l'ajoute à la ligne concernant le bien en question
        row2 = pd.concat([row, pd.Series(new_value, index=["estimated"])])
        # On transforme notre Serie en dataframe.
        row_df = pd.DataFrame([row2], columns=row2.index)
        # On prédit le prix de vente du bien
        row_predict = tree_local.predict(row_df)
        # On ajoute la prédiction à la liste de prédictions
        predictions.append(row_predict[0])
    
    elif type_local == "Maison":
        # On enlève la variable "Type local"
        row.drop("Type local", axis=0, inplace=True)
        # On enlève la variable "Nombre de lots"
        row.drop("Nombre de lots", axis=0, inplace=True)
        # On calcule une estimation du prix du bien
        new_value = row["Surface reelle bati"] * row["q3_prixm2"]
        # On l'ajoute à la ligne concernant le bien en question
        row2 = pd.concat([row, pd.Series(new_value, index=["estimated"])])
        # On transforme notre Serie en dataframe.
        row_df = pd.DataFrame([row2], columns=row2.index)
        # On réalise une AFDM sur nos données
        row_trans  = preprocessing_maison.transform(row_df)
        # On prédit le prix de vente du bien
        row_predict = regression_maison.predict(row_trans)
        # On ajoute la prédiction à la liste de prédictions
        predictions.append(row_predict[0])
    
    elif type_local == "Appartement":
        # On enlève la variable "Type local"
        row.drop("Type local", axis=0, inplace=True)
        # On calcule une estimation du prix du bien
        new_value = row["Surface reelle bati"] * row["q3_prixm2"]
        # On l'ajoute à la ligne concernant le bien en question
        row2 = pd.concat([row, pd.Series(new_value, index=["estimated"])])
        # On transforme notre Serie en dataframe.
        row_df = pd.DataFrame([row2], columns=row2.index)
        # On réalise une AFDM sur nos données
        row_trans  = preprocessing_appartement.transform(row_df)
        # On prédit le prix de vente du bien
        row_predict = regression_appartement.predict(row_trans)
        # On ajoute la prédiction à la liste de prédictions
        predictions.append(row_predict[0])
    
    elif type_local == "Dépendance":
        # On enlève la variable "Type local"
        row.drop("Type local", axis=0, inplace=True)
        # On enlève la variable "Surface relle bati"
        row.drop("Surface reelle bati", axis=0, inplace=True)
        # On transforme notre Serie en dataframe.
        row_df = pd.DataFrame([row], columns=row.index)
        # On réalise une AFDM sur nos données
        row_trans  = preprocessing_dep.transform(row_df)
        # On prédit le prix de vente du bien
        row_predict = regression_dep.predict(row_trans)
        # On ajoute la prédiction à la liste de prédictions
        predictions.append(row_predict[0])

In [388]:
print(predictions[:50])


[76893.5828592138, 219010.68337436777, 229215.29438866806, 192895.4724058702, 223646.75541981694, 156146.82484170748, 156146.82484170748, 186269.15318131424, 295793.7027269209, 161382.7951039307, 200695.87475234768, 209155.8710545299, 267668.1655297447, 191475.59774723326, 221232.69413296552, 173962.40115153886, 256902.0319965361, 174506.97804707757, 77916.80474813296, 137968.04193548387, 191351.02103448275, 220373.07954085388, 194986.00709957423, 226015.04580635967, 236889.66536875712, 196541.63833029338, 224815.2911783638, 296649.89195678686, 294173.85692880827, 224232.66224395108, 239150.1935587977, 255968.58280032955, 155570.9262613807, 211903.67126374628, 286308.43803793256, 174172.10079118639, 203260.7604130516, 216016.42309128158, 265606.9413368236, 225510.5377818315, 286586.7827288095, 156146.82484170748, 134284.3884597735, 211574.36611969522, 156146.82484170748, 381909.09847860946, 221021.57876980954, 157608.04375608443, 215639.36215665026, 167169.57438214374]


In [389]:
# On s'assure que notre liste de prédictions est de la même taille que notre échantillon d'apprentissage
len(predictions)

378041

In [384]:
sub.shape

(378041, 35)

In [374]:
# On transforme notre liste de prédictions en dataframe
pred_df = pd.DataFrame(predictions)
# Avant de l'enregistrer en csv pour pouvoir l'envoyer sur la compétition Kaggle
pred_df.to_csv("predictions_kaggle.csv", index=False, header=False)